# Teoría de la Correlación en Minería de Datos

La correlación es una herramienta estadística fundamental para entender cómo se relacionan dos variables. Se utiliza en minería de datos, machine learning, análisis exploratorio y ciencia de datos para identificar patrones, dependencias, redundancias, multicolinealidad y posibles causas subyacentes.

Este cuaderno presenta una teoría extensa y estructurada sobre la correlación, acompañada de ejemplos prácticos en Python.

## 1. ¿Qué es la correlación?
La correlación mide la fuerza y la dirección de la relación entre dos variables numéricas.

*   **Dirección:** positiva o negativa
*   **Fuerza:** débil, moderada o fuerte
*   **Tipo de relación:** lineal o monótona (no necesariamente lineal)

> **Nota:** La correlación NO implica causalidad, pero sí puede ser indicio útil para plantear hipótesis, modelar variables o detectar redundancias.

## 2. Signo y magnitud de la correlación

### Dirección
*   **Positiva (r > 0):** cuando una variable aumenta, la otra también.
*   **Negativa (r < 0):** cuando una variable aumenta, la otra disminuye.
*   **Cero (r = 0):** no hay relación lineal observable.

### Magnitud (reglas generales)

| Valor r | Interpretación |
| :--- | :--- |
| 0.0 – 0.2 | Muy débil (Relación mínima) |
| 0.2 – 0.4 | Débil (Relación pequeña) |
| 0.4 – 0.6 | Moderada (Relación clara) |
| 0.6 – 0.8 | Fuerte (Asociación notable) |
| 0.8 – 1.0 | Muy fuerte (Relación casi perfecta) |

*Estas reglas varían según la disciplina (psicología, economía, ingeniería, ciencias naturales).*

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Configuración de gráficas
sns.set(style="whitegrid")
%matplotlib inline

## 3. Tipos de correlación

### 3.1. Correlación de Pearson (r)
Mide la relación lineal entre dos variables continuas.

**Supuestos:**
*   Relación aproximadamente lineal
*   Variables continuas
*   Sin outliers extremos (muy sensibles)
*   Aproximadamente normales

**¿Cuándo usarla?**
*   Cuando la gráfica de dispersión sugiere linealidad.
*   Cuando estás construyendo modelos lineales, regresión múltiple o PCA.

In [ ]:
# Ejemplo: Correlación de Pearson
np.random.seed(42)
x = np.linspace(0, 10, 100)
y_linear = 2 * x + np.random.normal(0, 1, 100)  # Relación lineal fuerte

df_pearson = pd.DataFrame({'X': x, 'Y': y_linear})
r_pearson = df_pearson.corr(method='pearson').iloc[0, 1]

plt.figure(figsize=(6, 4))
sns.scatterplot(x='X', y='Y', data=df_pearson)
plt.title(f'Correlación de Pearson: r = {r_pearson:.2f}')
plt.show()

### 3.2. Correlación de Spearman (ρ)
Mide la relación monótona, usando el rango de los datos.

**Características:**
*   No requiere normalidad
*   Mucho más robusta a outliers
*   Funciona bien con variables ordinales
*   Captura relaciones monótonas no lineales

**¿Cuándo usarla?**
*   Cuando la relación no es lineal pero sí monótona (creciente o decreciente).
*   Cuando hay outliers o datos muy sesgados.

In [ ]:
# Ejemplo: Correlación de Spearman (Relación Monótona no lineal)
y_monotonic = x**3  # Relación cúbica (monótona pero no lineal)

df_spearman = pd.DataFrame({'X': x, 'Y': y_monotonic})
r_spearman = df_spearman.corr(method='spearman').iloc[0, 1]
r_pearson_comp = df_spearman.corr(method='pearson').iloc[0, 1]

plt.figure(figsize=(6, 4))
sns.scatterplot(x='X', y='Y', data=df_spearman)
plt.title(f'Spearman: {r_spearman:.2f} vs Pearson: {r_pearson_comp:.2f}')
plt.show()

### 3.3. Correlación de Kendall (τ)
Basada en pares concordantes y discordantes entre dos variables.

**Características:**
*   Más robusta que Spearman
*   Ideal para datos ordinales
*   Más interpretativa cuando hay empates

**¿Cuándo usarla?**
*   Con datasets pequeños
*   Con valores ordinales
*   Cuando se quiere comparar la probabilidad de que dos observaciones mantengan el mismo orden

In [ ]:
# Ejemplo: Comparación rápida
df_kendall = pd.DataFrame({'A': [1, 2, 3, 4, 5], 'B': [1, 3, 2, 4, 6]})
tau = df_kendall.corr(method='kendall').iloc[0, 1]
print(f"Correlación de Kendall: {tau:.2f}")

## 4. Visualización de relaciones
Antes de calcular correlaciones, **SIEMPRE** se recomienda visualizar.

**Herramientas:**
*   **Scatter plots:** para evaluar linealidad
*   **Pairplots:** relaciones múltiples
*   **Heatmaps:** matrices de correlación
*   **Diagramas de cajas:** detectar outliers

La visualización puede mostrar relaciones que los coeficientes no captan (curvilíneas, agrupaciones, efectos de confusores).

In [ ]:
# Generación de datos sintéticos con múltiples variables
data = pd.DataFrame(
    np.random.randn(100, 4),
    columns=['Var1', 'Var2', 'Var3', 'Var4']
)
data['Var2'] = data['Var1'] * 0.8 + np.random.normal(0, 0.5, 100)  # Corelacionada con Var1

# Heatmap de correlación
plt.figure(figsize=(8, 6))
sns.heatmap(data.corr(), annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title("Matriz de Correlación (Heatmap)")
plt.show()

## 5. Factores que distorsionan la correlación

### 5.1 Outliers
Pueden aumentar o disminuir drásticamente la correlación de Pearson. Spearman y Kendall son más resistentes.

### 5.2 No linealidad
Variables pueden tener relación curvilínea (ej.: U invertida) con r ≈ 0.
*   **Ejemplo clásico:** relación entre rendimiento y cantidad de entrenamiento.
*   Podría ser fuerte, pero no lineal → Pearson falla.

### 5.3 Heterocedasticidad
Si la dispersión aumenta con la variable:
*   Pearson pierde fiabilidad
*   Puede ser necesario transformar datos (log, box-cox)

In [ ]:
# Impacto de Outliers
x_out = np.append(x[:20], [50]) # Agregamos un outlier extremo
y_out = np.append(y_linear[:20], [0])

df_out = pd.DataFrame({'X': x_out, 'Y': y_out})
r_pearson_out = df_out.corr(method='pearson').iloc[0, 1]
r_spearman_out = df_out.corr(method='spearman').iloc[0, 1]

plt.figure(figsize=(6, 4))
sns.scatterplot(x='X', y='Y', data=df_out)
plt.title(f'Con Outlier -> Pearson: {r_pearson_out:.2f}, Spearman: {r_spearman_out:.2f}')
plt.show()

## 6. Correlación ≠ causalidad
Este es uno de los errores más comunes. Una correlación NO significa causalidad debido a:

*   Coincidencia aleatoria
*   Causa inversa
*   Variable oculta o confusora (spurious correlation)
*   Relación no causal pero asociada por un tercer fenómeno

**Ejemplo clásico:** Helados y ahogamientos correlacionan. Causa común: el calor del verano.

## 7. Correlación en minería de datos y machine learning
**Usos comunes:**
*   Selección de características (Feature Selection)
*   Detección de multicolinealidad
*   Ingeniería de variables
*   Evaluación de dependencia
*   Análisis exploratorio previo al modelado

## 8. Multicolinealidad
En modelos que dependen de relaciones lineales (regresión, GLM), si dos variables están muy correlacionadas:

*   Inflan los errores estándar
*   Dificultan la interpretación
*   Causan inestabilidad numérica

**Indicador más usado: VIF (Variance Inflation Factor)**
*   VIF > 5 → sospechoso
*   VIF > 10 → problemática seria

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# Calcular VIF para el dataframe 'data' creado anteriormente
X = add_constant(data)
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]

display(vif_data)

## 9. Correlación parcial
Mide la correlación entre X e Y controlando el efecto de una tercera variable Z.

**Útil para distinguir relaciones “reales” de las que solo existen por un confusor.**

In [ ]:
# Ejemplo conceptual de correlación parcial (usando pingouin si estuviera disponible, aqui simulado con pandas)
# Si Z es la causa común de X e Y, la correlación parcial entre X e Y dado Z debería ser cercana a 0.

z = np.random.normal(0, 1, 100)
x_c = 0.8 * z + np.random.normal(0, 0.5, 100)
y_c = 0.8 * z + np.random.normal(0, 0.5, 100) # X e Y correlacionados por causa de Z

df_conf = pd.DataFrame({'X': x_c, 'Y': y_c, 'Z': z})
print("Correlación bruta X-Y:", df_conf.corr().loc['X', 'Y'])

# Para eliminar el efecto de Z, podríamos usar regresiones (residuos) o librerías específicas (como pingouin.partial_corr)
# import pingouin as pg
# pg.partial_corr(data=df_conf, x='X', y='Y', covar='Z')
print("Se esperaría que la correlación parcial (quitando Z) baje considerablemente")

## 10. Correlación con variables categóricas
Para combinaciones entre distintos tipos de variables:

| Tipo de variables | Método sugerido |
| :--- | :--- |
| **Numérica - Numérica** | Pearson, Spearman, Kendall |
| **Numérica - Categórica** | ANOVA, correlación punto-biserial, dummies + Pearson |
| **Categórica - Categórica** | Chi-cuadrado, V de Cramer |

## 11. Interpretación práctica y Respuestas a Preguntas Clave

A continuación, respondemos las preguntas clave para interpretar correctamente la correlación en sus análisis:

### **1. ¿La relación es lineal?**
**Respuesta:** No asuma linealidad solo con el coeficiente. **Visualice** siempre (scatter plot). Si ve una curva (como una parábola), la correlación de Pearson podría ser 0 aunque haya una relación fuerte. En esos casos, use Spearman o transforme las variables.

### **2. ¿Hay outliers que la distorsionan?**
**Respuesta:** Los valores atípicos pueden inflar o destruir una correlación de Pearson. Revise mediante **Diagramas de Caja (Boxplots)** o gráficos de dispersión. Si existen, considere usar Spearman o evaluar si el outlier es un error de datos o un evento significativo.

### **3. ¿Existe un confusor?**
**Respuesta:** Si dos variables correlacionan extrañamente (ej. ventas de helado y ataques de tiburones), busque una tercera variable (el clima/verano) que explique ambas. Use correlación parcial o sentido común para no inferir causalidad errónea.

### **4. ¿Es razonable desde el dominio del problema?**
**Respuesta:** La estadística debe ir de la mano con el conocimiento del negocio. Si encuentra una correlación del 99% entre variables que no deberían tener relación, dude de los datos (data leakage, errores de medición).

### **5. ¿Tiene implicación para el modelo?**
**Respuesta:**
*   **Alta correlación entre features (Multicolinealidad):** Es malo para modelos lineales (Regresión, Logística) porque hace inestables los coeficientes. Considere eliminar una de las variables o usar técnicas de reducción de dimensión (PCA).
*   **Correlación con el target:** Es lo que buscamos. Variables con alta correlación con la variable objetivo son buenos predictores.